In [2]:
import pandas as pd
import json
import openai
import inspect
from dotenv import load_dotenv
import os
load_dotenv("..\credentials.env",override=True)

True

In [3]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential

In [7]:
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
subscription_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = "gpt-4o"

# Azure AI Search setup
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT"); # Add your Azure AI Search endpoint here
search_key = os.getenv("AZURE_SEARCH_KEY"); # Add your Azure AI Search admin key here
search_index_name = "northwind-vectorstore"

state_messages = []

In [5]:
client = openai.AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)

search_client = SearchClient(
     endpoint=search_endpoint,
     index_name=search_index_name,    
     credential=AzureKeyCredential(search_key)
 )

In [8]:
# Provide instructions to the model
GROUNDED_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that is not contained in the retrieved information from provided sources.
"""

#

In [11]:

def advanced_rag_OYD(user_query, system_msg = GROUNDED_PROMPT, reset_history=False):
    
    if(reset_history):
        state_messages.clear()
        state_messages.append({"role": "system", "content": system_msg})

    state_messages.append({"role": "user", "content": user_query})

    messages=[
            {"role": "user", "content": m["content"]}
            for m in state_messages
    ]

    # Generate the completion  
    response = client.chat.completions.create(  
        model=deployment,  
        messages=messages,  
        max_tokens=800,  
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,  
        stop=None,  
        stream=False, #no streaming     
        extra_body={
        "data_sources": [{
            "type": "azure_search",
            "parameters": {
                "filter": None,
                "endpoint": f"{search_endpoint}",
                "index_name": search_index_name,
                "semantic_configuration": "northwind-vectorstore-semantic-configuration",
                "authentication": {
                "type": "api_key",
                "key": f"{search_key}"
                },
                "embedding_dependency": {
                "type": "endpoint",
                "endpoint": f"{endpoint}/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview",
                "authentication": {
                    "type": "api_key",
                    "key": subscription_key
                }
                },
                "query_type": "vector_simple_hybrid",
                "in_scope": True,
                "role_information": "You are an AI assistant that helps people find information based on retrieved data.\n",
                "strictness": 3,
                "top_n_documents": 5
            }
            }]
        }
    )
    print(response.choices[0].message.content)

In [12]:
advanced_rag_OYD("Does Northwind health plan covers eye exams?")

Yes, the Northwind health plan covers vision services, which would include eye exams [doc1][doc4].


In [13]:
advanced_rag_OYD("I am unable to listen to the sound, does northwind cover the test?")

Yes, the Northwind Standard plan covers hearing tests and evaluations as part of its comprehensive health plan, which includes medical, vision, and dental services [doc1][doc2]. The plan provides coverage for routine hearing aid maintenance and repair, as well as counseling and hearing aid evaluations [doc2].


In [14]:
advanced_rag_OYD("what should I do if I feel feverish?")

The requested information is not available in the retrieved data. Please try another query or topic.


In [15]:
advanced_rag_OYD("how can I improve my health?")

To improve your health, you can consider the following tips:

1. **Get enough sleep:** Ensure that you are getting adequate rest each night as it helps your body to recover and stay energized [doc2].
2. **Eat a balanced diet:** Incorporate a variety of fruits, vegetables, lean proteins, and whole grains into your meals to maintain a healthy diet [doc2].
3. **Exercise regularly:** Aim for at least 30 minutes of physical activity each day to help burn calories and improve your physical health [doc2].
4. **Drink plenty of water:** Staying hydrated is important for overall health and can assist with weight management [doc2].
5. **Avoid sugary drinks:** Try to reduce or eliminate the consumption of sodas and energy drinks that add extra calories to your diet [doc2].
6. **Track your progress:** Keep a record of your health goals and monitor your progress to stay motivated and on track [doc2].
7. **Seek support:** Surround yourself with a support system, including friends, family, or healthca

In [16]:
advanced_rag_OYD("Does Northwind plans cover testing cholestrol levels?")


Yes, Northwind health plans cover testing for cholesterol levels. This service is included as part of the preventive care services under the Northwind Standard plan [doc1].
